In [ ]:
%matplotlib inline
import pandas as pd
import os
import networkx as nx
import igraph as ig
import numpy as np
import louvain
import matplotlib.pyplot as plt

In [ ]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'
links_df = pd.read_csv(os.path.join(filepath, 'case-to-article-links.csv'))

In [ ]:
print(links_df.shape, links_df.drop_duplicates().shape)

In [ ]:
links_df.head()

In [ ]:
case_nodes = links_df.source.unique()
art_nodes = links_df.target.unique()

In [ ]:
nr_cases = links_df.source.nunique()
nr_articles = links_df.target.nunique()
print(nr_cases, nr_articles)

In [ ]:
links_df.linktype.value_counts()

In [ ]:
g = nx.from_pandas_edgelist(links_df, source='source', target='target')

In [ ]:
nx.set_node_attributes(g, {n: 'case' for n in case_nodes}, name='type')
nx.set_node_attributes(g, {n: 'article' for n in art_nodes}, name='type')

In [ ]:
print(nx.info(g))

In [ ]:
# Convert to iGraph
g_ig = ig.Graph.TupleList(g.edges())

att_list = set(np.array([list(d.keys()) for n, d in g.nodes(data=True)]).flatten())

for att in att_list:
    att_dict = nx.get_node_attributes(g, att)
    g_ig.vs[att] = [att_dict[n] for n in g_ig.vs['name']]

In [ ]:
ccs = list(nx.connected_components(g))

In [ ]:
ccs_sizes = np.array([len(c) for c in ccs])
print("Number of connected components:", len(ccs))
print("Relative size of largest component:", np.max(ccs_sizes)/np.sum(ccs_sizes))

## Community detction

In [ ]:
p_01, p_0, p_1 = louvain.CPMVertexPartition.Bipartite(g_ig, resolution_parameter_01=0.01)
p_01.summary()

In [ ]:
optimiser = louvain.Optimiser()
diff = optimiser.optimise_partition_multiplex([p_01, p_0, p_1], layer_weights=[1, -1, -1])

In [ ]:
g_ig.vs['community'] = p_01.membership

In [ ]:
p_01.summary()

In [ ]:
p_0.summary()

In [ ]:
len(p_0.membership)

In [ ]:
cluster_df = pd.DataFrame({'name': g_ig.vs['name'], 'community': g_ig.vs['community'],  'type': g_ig.vs['type']})

In [ ]:
cluster_df.to_csv(os.path.join(filepath, 'bimodal_clusters.csv'), index=False)

In [ ]:
comm_counts = cluster_df.groupby(['community', 'type']).count()['name'].unstack()
comm_counts[:20]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 20))

n = 50

comm_counts[:n].plot(kind='bar', ax=axes[0], title='Cases and articles')
comm_counts['article'][:n].plot(kind='bar', ax=axes[1], title='Articles')
comm_counts['case'][:n].plot(kind='bar', ax=axes[2], title='Cases')

In [ ]:
# Merge with nodes
art_nodes = pd.read_csv(os.path.join(filepath, 'article_nodes_nodup_min5.csv'))
case_nodes = pd.read_csv(os.path.join(filepath, 'case_nodes_simple.csv'))

In [ ]:
case_nodes = case_nodes.merge(cluster_df[cluster_df['type']=='case'].drop('type', axis=1), how='left', left_on='lido_id', right_on='name', suffixes=('', '_bimodal'))
case_nodes = case_nodes.drop('name', axis=1).rename(columns={'community': 'community_bimodal'})

In [ ]:
case_nodes.head()

In [ ]:
art_nodes = art_nodes.merge(cluster_df[cluster_df['type']=='article'].drop('type', axis=1), how='left', left_on='id', right_on='name', suffixes=('', '_bimodal')).drop('name', axis=1)

In [ ]:
art_nodes.to_csv(os.path.join(filepath, 'article_nodes_nodup_min5_bimodal.csv'), index=False)
case_nodes.to_csv(os.path.join(filepath, 'case_nodes_simple_bimodal.csv'), index=False)

In [ ]:
ctab = pd.crosstab(art_nodes['community'], art_nodes['community_bimodal'])
ctab.shape

In [ ]:
im = plt.imshow(ctab.as_matrix(), aspect='equal')

In [ ]:
# Which articles are cited in the largest community?
art_nodes[art_nodes.community_bimodal==0][['label', 'community']]